In [1]:
from transformers import BertModel

model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [3]:
# set question
question = "Quando ocorreu a octagésima reunião do conselho diretor do instituto de informática?"

In [4]:
# open corpus
def openFile(path):
    # open file path
    my_file = open(path, encoding='utf-8')
    # for each line in the file
    file_var = ""
    for line in my_file:
        # append line to variable
        file_var += line
    # return variable
    return file_var

corpus = openFile("D:\\Documentos\\Tutorials\\MachineLearning\\tensorflow\\docs\\txt\\80a Reuni+úo.txt")
corpus

' Ata da 80ª (octogésima) reunião do Conselho Diretor do Instituto de Informática da Universidade Federal de Goiás.. Atas das 78ª e 79ª reuniões ordinárias e das reuniões extraordinárias dos dias 19/04/2006 e 07/06/2006.. Indicação de professor para trabalhar junto a PRODIRH na implantação de software  livre  na  UFG  –  solicitação  do  prof.  Hugo  Alexandre  D.  do Nascimento.. Indicação de professor responsável por evento de software livre durante o Conpeex.. Regulamento de Projeto Final de Curso 1 e 2 – prof. Hugo Alexandre Dantas do Nascimento.. Regulamento Estágio do INF – profa. Nilzete Olímpio Álvares.. Critérios para qualificação do corpo docente – profs. Rommel, Fábio Moreira e Cristiane.. Projetos de Extensão – profs. Cristiane e Sean. Outros. Às quartoze horas e vinte minutos do dia vinte e um de junho do ano de dois mil e seis, na sala 111 do Instituto de Informática, reuniram-se os professores abaixo relacionados para a realização de sessão ordinária do Conselho Diretor 

In [5]:
# create a questions and answers pipeline for the model and t he questions/answers
import torch
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    return 'Answer: "' + answer + '"'

In [6]:
# format corpus for the model
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80)

f_corpus = wrapper.fill(corpus)
print(f_corpus)

 Ata da 80ª (octogésima) reunião do Conselho Diretor do Instituto de Informática
da Universidade Federal de Goiás.. Atas das 78ª e 79ª reuniões ordinárias e das
reuniões extraordinárias dos dias 19/04/2006 e 07/06/2006.. Indicação de
professor para trabalhar junto a PRODIRH na implantação de software  livre  na
UFG  –  solicitação  do  prof.  Hugo  Alexandre  D.  do Nascimento.. Indicação
de professor responsável por evento de software livre durante o Conpeex..
Regulamento de Projeto Final de Curso 1 e 2 – prof. Hugo Alexandre Dantas do
Nascimento.. Regulamento Estágio do INF – profa. Nilzete Olímpio Álvares..
Critérios para qualificação do corpo docente – profs. Rommel, Fábio Moreira e
Cristiane.. Projetos de Extensão – profs. Cristiane e Sean. Outros. Às quartoze
horas e vinte minutos do dia vinte e um de junho do ano de dois mil e seis, na
sala 111 do Instituto de Informática, reuniram-se os professores abaixo
relacionados para a realização de sessão ordinária do Conselho Diretor do

In [7]:
# run question on cf_corpus
answer_question(question, corpus)

Query has 1,028 tokens.



RuntimeError: The size of tensor a (1028) must match the size of tensor b (512) at non-singleton dimension 1